In [1]:
import pandas as pd, numpy as np, scipy, os, itertools, sys
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
sys.path.append('..')
from func import loaders
from import_library import *

In [2]:
outcome_df = pd.read_csv('../data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']]
data_type ='mean' #mean, var
ld_excel = loaders(data_type = data_type)
ld_mat = loaders(data_type=data_type)
raw_data_path = '/Volumes/DatasetCGRD/SCD/data/image data/dFC matrix matlab/'
for next_path in sorted(os.listdir(raw_data_path)):
    if 'C0018FC_z.mat' not in next_path:
        ld_mat.load_mat(data_path = os.path.join(raw_data_path,next_path))
alff_index, alff_value, alff_columns = ld_excel.load_csv(data_path=f'../data/mdALFF matrix excel/mdALFF_{data_type}.csv')
fc_value = ld_mat.callback()

In [3]:
model_stack_ALFF = [None, None]
model_stack_FC = [None, None]
model_feature_stack = []
for idx, outcome_name in enumerate(['AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']):
# for idx, outcome_name in enumerate(['AD8主觀認知障礙[0-1,>=2]']):
    forest = RandomForestRegressor(max_depth = 5, n_jobs=-1, 
                        random_state=123)
    # FC feature selection
    print(f"FC {data_type} Feature Selection:", outcome_name)
    print(outcome_name)
    feat_selector_fc = BorutaPy(forest, verbose=0, random_state=123, max_iter=10)
    feat_selector_fc.fit(fc_value, np.array(outcome_df[outcome_name]))
    model_stack_FC[idx] = (feat_selector_fc)
    ranking_stack = []
    for (value, colum) in zip(feat_selector_fc.ranking_, [f'FC_{data_type}_{i}' for i in range (fc_value.shape[1])]):
        ranking_stack.append([value, colum])
    ranking_stack = sorted(ranking_stack)
    pd_rs = pd.DataFrame(ranking_stack, columns=['rank', 'feature_location'])
    pd_rs.to_csv(f'./dataset/{outcome_name}_{data_type}_fc.csv', index=False)
    model_feature_stack.append(pd_rs)

    # # ALFF feature selection
    # print(f"ALFF {data_type} Feature Selection:", outcome_name)
    # feat_selector_alff = BorutaPy(forest, verbose=0, random_state=123, max_iter=10, alpha=0.01, two_step=True)
    # feat_selector_alff.fit(np.array(alff_value), np.array(outcome_df[outcome_name]))
    # model_stack_ALFF[idx] = feat_selector_alff

    # ranking_stack = []
    # for (value, colum) in zip(feat_selector_alff.ranking_,alff_value.columns.tolist()):
    #     ranking_stack.append([value, colum])
    # ranking_stack = sorted(ranking_stack)
    # pd_rs = pd.DataFrame(ranking_stack, columns=['rank', 'feature_location'])
    # pd_rs.to_csv(f'./dataset/{outcome_name}_{data_type}_overall.csv', index=False)
    # model_feature_stack.append(pd_rs)

FC mean Feature Selection: AD8主觀認知障礙[0-1,>=2]
AD8主觀認知障礙[0-1,>=2]
FC mean Feature Selection: MOCA客觀認知測驗分數[<=23,>=24]
MOCA客觀認知測驗分數[<=23,>=24]


In [4]:
for i in zip(model_feature_stack, ['AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']):
    print(i[1], data_type)
    print(i[0].head(30))
    print('\n')

AD8主觀認知障礙[0-1,>=2] mean
    rank feature_location
0      2    FC_mean_23144
1      2    FC_mean_36058
2      2    FC_mean_39614
3      3    FC_mean_29892
4      4     FC_mean_1650
5      5    FC_mean_17588
6      5    FC_mean_39295
7      8    FC_mean_27356
8      8    FC_mean_30439
9      8    FC_mean_37665
10    10     FC_mean_3501
11    11    FC_mean_22649
12    12    FC_mean_43425
13    13    FC_mean_16908
14    14    FC_mean_22795
15    15    FC_mean_28991
16    16    FC_mean_42558
17    17    FC_mean_30944
18    18    FC_mean_29673
19    18    FC_mean_39380
20    20    FC_mean_27225
21    20    FC_mean_42975
22    22    FC_mean_17522
23    23    FC_mean_39606
24    24    FC_mean_27359
25    25    FC_mean_24034
26    26    FC_mean_17269
27    27    FC_mean_37054
28    28    FC_mean_41667
29    29    FC_mean_22650


MOCA客觀認知測驗分數[<=23,>=24] mean
    rank feature_location
0      2     FC_mean_2995
1      3     FC_mean_3611
2      4     FC_mean_6495
3      5    FC_mean_14903
4      5 